In [28]:
import pandas as pd
import glob

path = "LoLesports_data"

all_files = glob.glob(path + "/*.csv")

df_list = []

for file in all_files[-3:]:
    temp_df = pd.read_csv(file, low_memory=False)
    df_list.append(temp_df)

df = pd.concat(df_list, ignore_index=True)
df = df[df["datacompleteness"] == "complete"]
df.drop(columns=["datacompleteness", "url", "year"], inplace=True)

train = df[(df["patch"] >= 12.01) & (df["patch"] <= 14.05)]
test = df[df["patch"] >= 14.06]

train.shape, test.shape

((266040, 158), (67380, 158))

In [29]:
teams = train[train["position"] == "team"].copy()
teams.drop(columns=["champion", "position"], inplace=True)
teams.shape

(44340, 156)

In [32]:
side_wins = teams.groupby(["patch", "league", "side"])["result"].sum().reset_index()

total_games = (teams.groupby(["patch", "league"])["result"].count() / 2).reset_index()
total_games.columns = ["patch", "league", "total_games"]

side_stats = pd.merge(side_wins, total_games, on=["patch", "league"])
side_stats["win_rate"] = side_stats["result"] / side_stats["total_games"]

side_stats

,patch,league,side,result,total_games,win_rate
0,12.01,CBLOL,Blue,6,10.0,0.600000
1,12.01,CBLOL,Red,4,10.0,0.400000
2,12.01,CBLOLA,Blue,6,10.0,0.600000
3,12.01,CBLOLA,Red,4,10.0,0.400000
4,12.01,DDH,Blue,10,24.0,0.416667
...,...,...,...,...,...,...
1657,14.05,PRM,Red,11,26.0,0.423077
1658,14.05,TCL,Blue,13,25.0,0.520000
1659,14.05,TCL,Red,12,25.0,0.480000
1660,14.05,VCS,Blue,13,23.0,0.565217


In [33]:
side_stats.to_csv("output/side_stats.csv", index=False)